### TODO

1. Fix where the temporary files are being stored.
2. Test with different sample data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pipulate import pip
import gap_analyzer_sauce
import nest_asyncio
nest_asyncio.apply()
import keys

job = "gapalyzer-02" # Give your session a unique name

In [ ]:
# --- ⚙️ Workflow Configuration ---
ROW_LIMIT = 3000  # Final Output row limit, low for fast iteration
COMPETITOR_LIMIT = 3  # Limit rows regardless of downloads, low for fast iteration
BROWSER_DOWNLOAD_PATH = "~/Downloads"  # The default directory where your browser downloads files
GLOBAL_WIDTH_ADJUSTMENT = 1.5  #Multiplier to globally adjust column widths (1.0 = no change, 1.2 = 20% wider)

print(f"✅ Configuration set: Final report will be limited to {ROW_LIMIT} rows.")
if COMPETITOR_LIMIT:
    print(f"✅ Configuration set: Processing will be limited to the top {COMPETITOR_LIMIT} competitors.")
else:
    print(f"✅ Configuration set: Processing all competitors.")

# Here are the Keys

In [ ]:

pip.api_key(job, key=keys.google)
botify_token = keys.botify


## Here are your Foes

### Save all of These

In [ ]:
import gap_analyzer_sauce # Import the new module

# Call the function from the sauce module
# This performs the extraction, stores domains via pip.set, prints URLs,
# and returns the domains list if needed elsewhere (though we primarily rely on pip state now).
competitor_domains = gap_analyzer_sauce.extract_domains_and_print_urls(job)

# Optional: You could add a pip.get here for verification if desired
# stored_domains = pip.get(job, 'competitor_domains', [])
# print(f"\nVerification: Retrieved {len(stored_domains)} domains from pip state.")

In [ ]:
import os
import shutil
from pathlib import Path
import glob

def collect_semrush_downloads(job: str, download_path_str: str, file_pattern: str = "*-organic.Positions*.xlsx"):
    """
    Moves downloaded SEMRush files matching a pattern from the user's download
    directory to a job-specific 'downloads/{job}/' folder within the Notebooks/
    directory.
    
    Args:
        job (str): The current job ID (e.g., "gapalyzer-01").
        download_path_str (str): The user's default browser download path (e.g., "~/Downloads").
        file_pattern (str): The glob pattern to match SEMRush files.
    """
    print("📦 Starting collection of new SEMRush downloads...")

    # 1. Define source and destination paths
    # Resolve the user's download path (handles ~)
    source_dir = Path(download_path_str).expanduser()
    
    # Define the destination path relative to the current working directory (Notebooks/)
    # This assumes the Notebook is run from the 'Notebooks' directory or its path is correct.
    destination_dir = Path("downloads") / job

    # 2. Create the destination directory if it doesn't exist
    destination_dir.mkdir(parents=True, exist_ok=True)
    print(f"Destination folder created/ensured: '{destination_dir.resolve()}'")

    # 3. Find files in the source directory matching the pattern
    # We use glob.glob for pattern matching, searching for both .xlsx and .csv
    files_to_move = []
    
    # Check for .xlsx files
    xlsx_files = glob.glob(str(source_dir / file_pattern))
    files_to_move.extend(xlsx_files)
    
    # Check for .csv files
    csv_pattern = file_pattern.replace(".xlsx", ".csv")
    csv_files = glob.glob(str(source_dir / csv_pattern))
    files_to_move.extend(csv_files)

    if not files_to_move:
        print("⚠️ No new files matching the pattern were found in the download directory. Skipping move.")
        return

    # 4. Move the files
    move_count = 0
    for source_file_path in files_to_move:
        source_file = Path(source_file_path)
        dest_file = destination_dir / source_file.name
        
        # Only move if the file doesn't already exist in the destination (to avoid overwriting)
        # This protects manually modified files, but new downloads will have unique timestamps anyway.
        if dest_file.exists():
             # Option: could log that it exists or decide to overwrite/rename. 
             # Given the SEMRush filename pattern contains a unique timestamp, we expect 
             # them to be new. Let's just avoid redundant logging.
             continue
        
        try:
            shutil.move(source_file, dest_file)
            print(f"  -> Moved: {source_file.name}")
            move_count += 1
        except Exception as e:
            print(f"  -> ❌ Error moving {source_file.name}: {e}")

    print(f"✅ Collection complete. {move_count} new files moved to '{destination_dir}'.")
    
    # --- Execute the function in the notebook ---
collect_semrush_downloads(job, BROWSER_DOWNLOAD_PATH)

In [ ]:
import nltk
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

In [ ]:
import itertools
from pathlib import Path
from IPython.display import display, Markdown

# NOTE: This cell assumes 'job' is defined (e.g., "gapalyzer-01")

# --- Define the file directory based on the job variable ---
semrush_gap_analysis_dir = Path("downloads") / job

# --- Combine glob results for both .xlsx and .csv ---
file_patterns = [
    "*-organic.Positions*.xlsx", 
    "*-organic.Positions*.csv"
]

# Use itertools.chain to efficiently combine generators from multiple glob calls
all_downloaded_files = sorted(list(itertools.chain.from_iterable(
    semrush_gap_analysis_dir.glob(pattern) for pattern in file_patterns
)))

# --- Display Results ---
if all_downloaded_files:
    # Use a Markdown block for formatted display with emoji
    markdown_output = ["## 💾 Found Downloaded Files"]
    markdown_output.append(f"✅ **{len(all_downloaded_files)} files** ready for processing in `{semrush_gap_analysis_dir}/`\n")
    
    for i, file in enumerate(all_downloaded_files):
        # The file name starts with the competitor's domain.
        try:
            # We strip the full file path name for cleaner display
            domain_name = file.name[:file.name.index("-organic.")].strip()
        except ValueError:
            # Fallback if the expected pattern is slightly off
            domain_name = file.name
            
        markdown_output.append(f"{i + 1}. **`{domain_name}`** ({file.suffix.upper()})")

    display(Markdown("\n".join(markdown_output)))
    
    # --- NEW FIX: Convert Path objects to strings for JSON serialization ---
    # The Pipulate core needs simple, JSON-serializable types (strings, lists, dicts, etc.)
    all_downloaded_files_as_str = [str(p) for p in all_downloaded_files]
    # ---------------------------------------------------------------------

    # For the next step, we'll store the list of files in the Pipulate pipeline.
    pip.set(job, 'semrush_files', all_downloaded_files_as_str)
    
else:
    display(Markdown(f"⚠️ **Warning:** No SEMRush files found in `{semrush_gap_analysis_dir}/`.\n(Looking for `*-organic.Positions*.xlsx` or `*.csv`)"))

In [ ]:
import pandas as pd
from tldextract import extract
import itertools
from pathlib import Path
from IPython.display import display

# --- SUPPORT FUNCTION (1-to-1 Transplant) ---
# NOTE: This function requires 'tldextract' to be installed (which you've handled).
def extract_registered_domain(url):
    """
    Extracts the registered domain (domain.suffix) from a URL/hostname.
    """
    extracted = extract(url)
    return f"{extracted.domain}.{extracted.suffix}"

# --- MAIN LOGIC ADAPTATION ---

# Variables required from previous Notebook cells:
# job, ROW_LIMIT, COMPETITOR_LIMIT, BROWSER_DOWNLOAD_PATH, client_domain, country_code
# semrush_gap_analysis_dir is assumed to be defined as Path("downloads") / job

# Define 'semrush_gap_analysis_dir' and 'semrush_lookup' based on prior context
# We use the 'job' variable to define the directory
semrush_gap_analysis_dir = Path("downloads") / job

# The client domain is the key for separating client vs. competitor data.
# We strip the full domain in case it contains a protocol or path.
# Assuming 'client_domain' is available from a keys/config cell (e.g., "example.com")
# Since we don't have 'client_domain' defined here, we'll temporarily define it for the port.
# Replace this line with proper import/assignment if moving to external module:
semrush_lookup = extract_registered_domain(keys.client_domain)

print(f"Creating a great big DataFrame for {semrush_lookup}...")

# 1. Adapt file globbing to handle BOTH CSV and XLSX (as done in the previous step)
file_patterns = ["*-organic.Positions*.xlsx", "*-organic.Positions*.csv"]
all_semrush_files = sorted(list(itertools.chain.from_iterable(
    semrush_gap_analysis_dir.glob(pattern) for pattern in file_patterns
)))

# Initialize data structures
cdict = {}
list_of_dfs = []
print("Loading SEMRush files: ", end="")

# 2. Loop through all found files
for j, data_file in enumerate(all_semrush_files):
    # Determine the file type and corresponding reader function
    is_excel = data_file.suffix.lower() == '.xlsx'
    read_func = pd.read_excel if is_excel else pd.read_csv
    
    # Original file name parsing logic
    nend = data_file.stem.index("-organic")
    xlabel = data_file.stem[:nend].replace("_", "/").replace("///", "://").strip('.')
    
    # Original domain extraction logic (using the locally defined function)
    just_domain = extract_registered_domain(xlabel)
    cdict[just_domain] = xlabel
    
    # Load data
    df = read_func(data_file)
    
    # Original column assignment logic
    if just_domain == xlabel:
        df["Domain"] = just_domain
    else:
        # Use the full X-label (e.g., sub.domain.com) if it's not just the registered domain
        df["Domain"] = xlabel
    
    # Original data segregation logic
    df["Client URL"] = df.apply(lambda row: row["URL"] if row["Domain"] == semrush_lookup else None, axis=1)
    df["Competitor URL"] = df.apply(lambda row: row["URL"] if row["Domain"] != semrush_lookup else None, axis=1)
    
    list_of_dfs.append(df)
    print(f"{j + 1} ", end="", flush=True)

print() # Newline after the loading count

if list_of_dfs:
    df2 = pd.concat(list_of_dfs)  # Concatenate like stacking CSVs
    
    # --- Original Excel Formatting Value Gathering ---
    # This logic appears to be for calculating Excel visual layout, 
    # but still needs to be run even if the formatting happens later.
    # It requires the 'bf.open_dir_widget' function to be defined or stubbed if not portable.
    # NOTE: Since `bf` is not defined, and `project_customizations`/`proceed` are missing, 
    # we must skip the non-portable lines to prevent breaking the Notebook.

    # Stubbing non-portable functions/logic to keep the structure intact
    # We remove the print statements related to bf/project/customization for now
    
    # The max_length calculation is fine to keep
    max_length = max(len(value) for value in cdict.values())
    row1_height = max_length * 7 # Unused variable for now, but ported
    
    rows, columns = df2.shape
    print()
    print(f"Rows: {rows:,}")
    print(f"Cols: {columns:,}")
    print()

    # NOTE: The subsequent conditional logic (lines 53-61 in the original)
    # involving `bf.open_dir_widget`, `project_customizations`, and `proceed()`
    # has been intentionally omitted here as it depends on external, undefined
    # modules (`bf`) and pipeline state (`project`, `project_customizations`, `proceed`)
    # that are not provided in the prompt's context and would cause the script to fail.
    # We only port the pure Pandas/Python logic.
    
    # The final output and pipeline update
    display(df2["Domain"].value_counts())
    
    # Store the result in the pipeline
    pip.set(job, 'semrush_master_df_json', df2.to_json(orient='records'))
    
else:
    print("Please put the CSVs in place.")

In [ ]:
import pandas as pd
from pathlib import Path
from IPython.display import display
from collections import defaultdict # Needed if cdict is modified to be a defaultdict

# --- PATH DEFINITIONS (Needed to replace external checks) ---
# Assumes 'job' is defined in a previous cell (e.g., "gapalyzer-01")
# Assumes 'df2' is the master DataFrame from the previous step
competitors_csv_file = Path("data") / f"{job}_competitors.csv"

# --- ADAPTED PIVOTING LOGIC ---

print("Pivoting data. Keyword count per competitor...\n")

# Original pivot operation
pivot_df = df2.pivot_table(index='Keyword', columns='Domain', values='Position', aggfunc='min')

# ORIGINAL LOGIC: pivot_df = bf.move_column_to_front(pivot_df, semrush_lookup)
# SURGICAL PORT: Use Pandas reindexing to move the column to the front.
if semrush_lookup in pivot_df.columns:
    cols = [semrush_lookup] + [col for col in pivot_df.columns if col != semrush_lookup]
    pivot_df = pivot_df[cols]
else:
    print(f"⚠️ Warning: Client domain '{semrush_lookup}' not found in pivot table columns.")


# Original: Get list of columns and calculate positioning
competitors = list(pivot_df.columns)
pivot_df['Competitors Positioning'] = pivot_df.iloc[:, 1:].notna().sum(axis=1)

# Original: Load or initialize df_competitors
if competitors_csv_file.exists():
    df_competitors = pd.read_csv(competitors_csv_file)
    df_competitors['Title'] = df_competitors['Title'].fillna('')
    df_competitors['Matched Title'] = df_competitors['Matched Title'].fillna('')
    print(f"✅ Loaded {len(df_competitors)} existing competitor records.")
else:
    # Use 'cdict' (created in the previous step) to initialize
    df_competitors = pd.DataFrame(list(cdict.items()), columns=['Domain', 'Column Label'])
    
    # Initialize 'Title' and 'Matched Title' columns explicitly
    df_competitors['Title'] = ''
    df_competitors['Matched Title'] = ''
    
    df_competitors.to_csv(competitors_csv_file, index=False)
    print(f"✅ Created new competitor file at '{competitors_csv_file}'.")

# Original: Print keyword counts per competitor (for debugging/visual confirmation)
counts = pivot_df.describe().loc['count']
# Ensure counts has data before proceeding with printing logic
if not counts.empty:
    max_digits = len(str(len(counts)))
    # Ensure all indices are strings for max length calculation
    max_index_width = max(len(str(index)) for index in counts.index) 
    
    # Ensure only non-NaN counts are considered for width calculation, fallback to 0 if all are NaN
    valid_counts = [count for count in counts if pd.notna(count)]
    max_count_width = max([len(f"{int(count):,}") for count in valid_counts] or [0])
    
    for i, (index, count) in enumerate(counts.items(), start=1):
        counter_str = str(i).zfill(max_digits)
        count_str = f"{int(count):,}" if pd.notna(count) else 'NaN'
        print(f"{counter_str}: {index:<{max_index_width}} - {count_str:>{max_count_width}}")
else:
    print("❌ No data to count after pivot table creation.")

# Original: Print rows and columns summary
rows, columns = df2.shape
rows2, columns2 = pivot_df.shape
print("\nThere is some natural deduping from pivot.\n")
print(f"Rows (master df): {rows:,}")
print(f"Rows (pivot df): {rows2:,} ({rows:,} - {rows2:,} = {rows - rows2:,} dupes removed.)")
print(f"Cols: {columns2:,}") # Use columns2 for the pivot_df column count

# Original: Display result
display(pivot_df)

# Store the final result in the pipeline
pip.set(job, 'keyword_pivot_df', pivot_df.to_json(orient='records'))
pip.set(job, 'competitors_df', df_competitors.to_json(orient='records'))

In [ ]:
import pandas as pd
import nest_asyncio
import asyncio
from pathlib import Path
from tldextract import extract
import wordninja
import httpx
import re
from collections import defaultdict # Already imported in a previous cell

# NOTE: This cell assumes 'job', 'semrush_lookup', 'df_competitors', 
#       and 'competitors_csv_file' are defined in prior cells.
# We also assume 'df_competitors' was loaded from 'competitors_csv_file' in the previous step.

# --- PATH DEFINITION FOR FILTER FILE ---
# Consolidating working files to the 'data' directory.
filter_file = Path("data") / f"{job}_filter_keywords.csv"


# --- REQUIRED SUPPORT FUNCTIONS (Surgically Ported from botifython.py) ---

# Headers and user_agent were defined globally in botifython.py, but are needed here for httpx
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
headers = {'User-Agent': user_agent}

def extract_registered_domain(url):
    """Extracts the registered domain (domain.suffix) from a URL/hostname."""
    extracted = extract(url)
    return f"{extracted.domain}.{extracted.suffix}"

def get_title_from_html(html_content):
    """Simple helper to extract the title from HTML content."""
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    title_tag = soup.find('title')
    return title_tag.text if title_tag else ''

def match_domain_in_title(domain, title):
    """Finds a stripped version of the domain in the title."""
    base_domain = domain.split('.')[0]
    pattern = ''.join([c + r'\s*' for c in base_domain])
    regex = re.compile(pattern, re.IGNORECASE)
    match = regex.search(title)
    if match:
        matched = match.group(0).strip()
        return matched
    return ''

async def async_check_url(url, domain, timeout):
    """Asynchronously checks a single domain and extracts title/matched title."""
    # Timeout is intentionally high (120s from the original context)
    try:
        async with httpx.AsyncClient(follow_redirects=True, headers=headers, timeout=timeout) as client:
            response = await client.get(url)
            if response.status_code == 200:
                if str(response.url) != url:
                    print(f"Redirected to {response.url} for {url}")
                title = get_title_from_html(response.text)
                matched_title = match_domain_in_title(domain, title)
                return str(response.url), title, matched_title, True
            else:
                print(f"Status Code {response.status_code} for {url}")
    except httpx.RequestError as e:
        print(f"Request failed for {url}: {str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred for {url}: {str(e)}")
    return url, None, None, False

def test_domains(domains, timeout=120):
    """Orchestrates async checks for a list of domains."""
    print(f"Giving up to {timeout} seconds to visit all sites...")
    tasks = [async_check_url(f'https://{domain}', domain, timeout) for domain in domains]
    results = asyncio.run(async_test_domains(domains, tasks))
    
    domain_results = {}
    for domain, result in zip(domains, results):
        # Handle exceptions gracefully as in the original bf.test_domains (part of the transplant)
        if isinstance(result, Exception):
            domain_results[domain] = {'url': None, 'title': None, 'matched_title': None}
        else:
            domain_results[domain] = {'url': result[0], 'title': result[1], 'matched_title': result[2]}
    return domain_results

async def async_test_domains(domains, tasks):
    """Internal helper for asyncio.gather."""
    return await asyncio.gather(*tasks, return_exceptions=True)

def split_domain_name(domain):
    """Splits a concatenated domain name into human-readable words (requires wordninja)."""
    words = wordninja.split(domain)
    return ' '.join(words)

# --- MAIN WORKFLOW LOGIC ---

print("Visiting competitor homepages for title tags for filters...\n")

# Original logic required to run async in Jupyter
nest_asyncio.apply()

# Lowercase existing matched titles for comparison
df_competitors['Matched Title'] = df_competitors['Matched Title'].str.lower()

# Find domains where 'Title' column is empty ('') or NaN
# Using .isna() on a string column returns False for '', so we check explicitly for the empty string
needs_titles = df_competitors[
    (df_competitors['Title'].isna()) | (df_competitors['Title'] == '')
].copy()

if not needs_titles.empty:
    # 1. Scrape Titles
    print(f"Gathering Titles for {len(needs_titles)} domains.")
    results = test_domains(needs_titles['Domain'].tolist())
    
    # 2. Prepare and Merge Data
    data_to_add = {
        'Domain': [],
        'Title': [],
        'Matched Title': []
    }
    
    for domain, info in results.items():
        data_to_add['Domain'].append(domain)
        data_to_add['Title'].append(info['title'] if info['title'] else '')
        data_to_add['Matched Title'].append(info['matched_title'] if info['matched_title'] else '')
    
    new_data_df = pd.DataFrame(data_to_add)
    
    # Use original combine_first logic for non-destructive update
    df_competitors.set_index('Domain', inplace=True)
    new_data_df.set_index('Domain', inplace=True)
    df_competitors = new_data_df.combine_first(df_competitors)
    df_competitors.reset_index(inplace=True)
    
    # Lowercase and persist the updated data
    df_competitors['Matched Title'] = df_competitors['Matched Title'].str.lower()
    df_competitors.to_csv(competitors_csv_file, index=False)
    print(f"✅ Updated competitor titles and saved to '{competitors_csv_file}'.")


# --- Create Keyword Filters ---

# Remove '.com' from both lists to create more generic keyword filters
extracted_domains = [extract_registered_domain(domain).replace('.com', '') for domain in df_competitors['Domain']]
matched_titles = [title.replace('.com', '') for title in df_competitors['Matched Title'].tolist() if title]

# Split domain names using wordninja (e.g., 'foobar' -> 'foo bar')
split_domains = [split_domain_name(domain) for domain in extracted_domains]

# Combine all lists, strip whitespace, and deduplicate
combined_list = [x.strip() for x in extracted_domains + matched_titles + split_domains if x]
combined_list = sorted(list(set(combined_list)))

# Persist to external filter file (allows user editing)
if not filter_file.exists():
    df_filter = pd.DataFrame(combined_list, columns=['Filter'])
    df_filter.to_csv(filter_file, index=False)
    print(f"✅ Created initial keyword filter file at '{filter_file}' for user editing.")
else:
    print(f"☑️ Keyword filter file already exists at '{filter_file}'. Skipping creation.")

# Store the final competitors DF in the pipeline
pip.set(job, 'competitors_df', df_competitors.to_json(orient='records'))

In [ ]:
import pandas as pd
from IPython.display import display

# NOTE: This cell assumes 'df2' (the result of the aggregation step) is available.

print("Adjusting SEMRush columns that were not part of competitor-columns pivot...")

# Assign aggregating function to each metric
# The chosen functions are critical for creating a single, best-case summary per keyword:
# - 'min' for Position: Gives the *best* rank achieved across all competitors who rank.
# - 'max' for Search Volume/Number of Results/Timestamp: Captures the highest value reported.
# - 'sum' for Traffic/Traffic Cost: Aggregates the total value across all competitor results.
# - 'mean' for Difficulty/CPC/Competition: Averages the difficulty/cost across all reporting sources.
# - 'first' for categorical data (URLs, Intents, SERP Features): Chooses the first encountered value.
agg_funcs = {
    'Position': 'min',
    'Search Volume': 'max',
    'CPC': 'mean',
    'Traffic': 'sum',
    'Traffic (%)': 'mean',
    'Traffic Cost': 'sum',
    'Keyword Difficulty': 'mean',
    'Previous position': 'first',
    'Competition': 'mean',
    'Number of Results': 'max',
    'Timestamp': 'max',
    'SERP Features by Keyword': 'first',
    'Keyword Intents': 'first',
    'Position Type': 'first',
    'URL': 'first',
    'Competitor URL': 'first',
    'Client URL': 'first'
}

# Apply the aggregation across the combined dataset (df2)
agg_df = df2.groupby('Keyword').agg(agg_funcs).reset_index()

# Add a derived metric: Keyword word count
agg_df['Number of Words'] = agg_df["Keyword"].apply(lambda x: len(x.split()))

# Drop the 'Position' column: It was only used for the pivot/min operation,
# but it's redundant/misleading now that the competitor position data is in pivot_df.
agg_df.drop(columns=['Position'], inplace=True)

print("Table of aggregates prepared.")

display(agg_df)

# Store the aggregated metrics in the pipeline
pip.set(job, 'keyword_aggregate_df_json', agg_df.to_json(orient='records'))

In [ ]:
import pandas as pd
from pathlib import Path
from IPython.display import display

# NOTE: This cell assumes 'job', 'pivot_df', 'agg_df', and 'filter_file' are defined in prior cells.

# --- PATH DEFINITION ---
# The filter file path is already defined in a previous step, but included here for clarity
filter_file = Path("data") / f"{job}_filter_keywords.csv"

# --- REQUIRED SUPPORT FUNCTION (Surgically Ported from botifython.py) ---

def reorder_columns_surgical(df, priority_column, after_column):
    """
    Surgical port of bf.reorder_columns: Moves a column immediately after a specified column.
    """
    if priority_column in df.columns and after_column in df.columns:
        columns = df.columns.drop(priority_column).tolist()
        after_column_index = columns.index(after_column)
        columns.insert(after_column_index + 1, priority_column)
        return df[columns]
    elif priority_column not in df.columns:
        print(f"⚠️ Warning: Priority column '{priority_column}' not found for reorder.")
    elif after_column not in df.columns:
        print(f"⚠️ Warning: After column '{after_column}' not found for reorder.")
    return df


print("Merging Pivot Data with Aggregate Data...")

# 1. Merge Pivot Data (Keyword as index) with Aggregate Data (Keyword as index/column)
pivotmerge_df = pd.merge(pivot_df.reset_index(), agg_df, on='Keyword', how='left')

print("Pivot and Aggregate Data Joined.\n")
rows, columns = pivotmerge_df.shape
print(f"Rows: {rows:,}")
print(f"Cols: {columns:,}")

# --- FILTERING LOGIC ---

print("\nBrand and Negative Filters being applied...")
# 2. Optionally Filter Brand & Negative Keywords
if filter_file.exists():
    df_filter = pd.read_csv(filter_file, header=0)
    
    # Ensure all list items are strings before joining into a regex pattern
    kw_filter = [str(f) for f in df_filter["Filter"].dropna().tolist()]
    
    if kw_filter:
        # Use re.escape to handle special characters in keywords and then join with '|' (OR)
        pattern = '|'.join([re.escape(keyword) for keyword in kw_filter])
        
        # Apply the filter: keep rows where Keyword DOES NOT contain the pattern
        filtered_df = pivotmerge_df[~pivotmerge_df["Keyword"].str.contains(pattern, case=False, na=False)]
        print(f"✅ Filter applied using {len(kw_filter)} terms from '{filter_file}'.")
    else:
        filtered_df = pivotmerge_df
        print("⚠️ Filter file exists but contains no terms. Skipping filter application.")
else:
    filtered_df = pivotmerge_df
    print(f"☑️ No filter file found at '{filter_file}'. Skipping negative filtering.")

rows_filtered, columns_filtered = filtered_df.shape
print(f"Rows: {rows_filtered:,} ({rows:,} - {rows_filtered:,} = {rows - rows_filtered:,} rows removed)")


# --- REORDERING AND FINAL POLISH ---

# 3. Apply Reordering Logic (Using the surgically defined function)
# NOTE: The original logic chains reorders based on previously moved columns.
temp_df = filtered_df.copy() # Use a temporary variable for clarity during chained operations

temp_df = reorder_columns_surgical(temp_df, "Search Volume", after_column="Keyword")
temp_df = reorder_columns_surgical(temp_df, "Number of Words", after_column="CPC")
temp_df = reorder_columns_surgical(temp_df, "CPC", after_column="Number of Words")
temp_df = reorder_columns_surgical(temp_df, "Number of Results", after_column="Position Type")
temp_df = reorder_columns_surgical(temp_df, "Timestamp", after_column="Number of Results")
temp_df = reorder_columns_surgical(temp_df, "Competitor URL", after_column="Client URL")

# 4. Final Arrange (Verbatim column ordering and sorting)
# The manual reorder logic below overrides the custom function, but we include it verbatim:
rest_of_columns = [col for col in temp_df.columns if col not in ['Keyword', 'Search Volume']]
new_column_order = ['Keyword', 'Search Volume'] + rest_of_columns

# The conditional block from the original (verbatim)
if 'Keyword' in temp_df.columns:
    temp_df = temp_df[['Keyword'] + ['Search Volume'] + [col for col in temp_df.columns if col not in ['Keyword', 'Search Volume']]]

# Apply the intended final order
filtered_df = temp_df[new_column_order]

# Final sorting and column drops
arranged_df = filtered_df.sort_values(by='Search Volume', ascending=False)
arranged_df.drop(columns=["Previous position", "Traffic", "Traffic (%)", "Traffic Cost"], inplace=True)

print("\nFinal Keyword Table Prepared.")

# Store the final result in the pipeline
pip.set(job, 'filtered_gap_analysis_df_json', arranged_df.to_json(orient='records'))

display(arranged_df)

In [ ]:
# --- BOTIFY API UTILITY FUNCTIONS (REQUIRES: requests, gzip, shutil, Path, time, pprint) ---
import requests
from time import sleep
import gzip
import shutil
from pprint import pprint

# Assumes 'keys' module with 'keys.botify' is imported in a previous cell

def download_file(download_url, output_path):
    response = requests.get(download_url, stream=True)
    if response.status_code == 200:
        output_path.parent.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
        with open(output_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        return True
    else:
        print(f"Failed to download file. Status Code: {response.status_code}")
        return False

def decompress_gz(gz_path, output_path):
    try:
        with gzip.open(gz_path, 'rb') as f_in, open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        print(f"Decompressed {output_path}")
        return True
    except Exception as e:
        print(f"Failed to decompress {gz_path}. Error: {e}")
        return False


def fetch_analysis_slugs(org, project, botify_token):
    """Fetch analysis slugs for a given project from the Botify API."""
    analysis_url = f"https://api.botify.com/v1/analyses/{org}/{project}/light"
    headers = {"Authorization": f"Token {botify_token}"}
    try:
        response = requests.get(analysis_url, headers=headers)
        response.raise_for_status()
        data = response.json()
        analysis_data = data.get('results', [])
        return [analysis['slug'] for analysis in analysis_data]
    except Exception as e:
        print(f"❌ Error fetching analysis slugs: {e}")
        return []


def export_data(version, org, project, export_payload, report_path, analysis=None, retry_url=None):
    """
    Unified function to export data using BQLv1 or BQLv2.
    version must be v1 or v2
    """
    file_base = report_path.stem
    path_base = Path(report_path).parent
    zip_name = path_base / f"{file_base}.gz"
    csv_name = Path(report_path)

    path_base.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists before proceeding

    if csv_name.exists():
        print(f"The file: {csv_name}")
        print("...already exists for analysis period. Exiting.")
        return (None, None)

    if zip_name.exists():
        print(f"☑️ {zip_name} found without corresponding CSV. Decompressing now...")
        decompress_success = decompress_gz(zip_name, csv_name)
        return (200, None) if decompress_success else (None, None)

    if retry_url:
        print(f"Using retry URL for direct download: {retry_url}")
        if download_file(retry_url, zip_name):  # Save as .gz file
            print("File downloaded successfully via retry URL.")
            if decompress_gz(zip_name, csv_name):  # Decompress .gz to .csv
                print("File decompressed successfully.")
                return (200, csv_name)
            else:
                print("Decompression failed.")
                return (None, None)
        else:
            print("Download failed using retry URL.")
            return (None, None)

    # Use the token from the keys module
    headers = {'Authorization': f'Token {keys.botify}', 'Content-Type': 'application/json'} 

    if version == 'v1':
        url = f'https://api.botify.com/v1/analyses/{org}/{project}/{analysis}/urls/export'
        response = requests.post(url, headers=headers, json=export_payload)
    else:  # version == 'v2'
        url = "https://api.botify.com/v1/jobs"
        response = requests.post(url, headers=headers, json=export_payload)

    if response.status_code not in [200, 201]:
        print(f"❌ Failed to start CSV export. Status Code: {response.status_code}.")
        print(response.reason, response.text)
        pprint(export_payload)
        return (response.status_code, None)

    export_job_details = response.json()
    job_url = export_job_details.get('job_url')
    if version == "v2":
        job_url = f'https://api.botify.com{job_url}'

    attempts = 300
    delay = 10
    print(f"{attempts} attempts will be made every {delay} seconds until download is ready...")

    while attempts > 0:
        sleep(delay)
        print(attempts, end=" ", flush=True)  # Countdown on the same line
        response_poll = requests.get(job_url, headers=headers)
        if response_poll.status_code == 200:
            job_status_details = response_poll.json()
            if job_status_details['job_status'] == 'DONE':
                print("\nExport job done.")
                download_url = job_status_details['results']['download_url']
                if download_file(download_url, zip_name):
                    print("File downloaded successfully.")
                    if decompress_gz(zip_name, csv_name):
                        print("File decompressed successfully.")
                        return (200, csv_name)
                    else:
                        print("Decompression failed.")
                        return ("Decompression failed 1.", None)
                else:
                    print("Download failed.")
                    return ("Download failed.", None)
            elif job_status_details['job_status'] == 'FAILED':
                print("\nExport job failed.")
                print(job_status_details.get('failure_reason', 'No failure reason provided.'))
                return ("Export job failed.", None)
        else:
            print(f"\nFailed to get export status. Status Code: {response_poll.status_code}")
            print(response_poll.text)

        attempts -= 1

    print("Unable to complete download attempts successfully.")
    return ("Unable to complete", None)

print("Loaded")

In [ ]:
import pandas as pd
from pathlib import Path
from IPython.display import display
from collections import defaultdict 
import requests
from time import sleep
import gzip
import shutil
from pprint import pprint

# --- 1. FIX: ROBUST PATH & VARIABLE DEFINITIONS ---
# This block dynamically finds the latest analysis slug, solving the 404 error.

# 1. Get URL from keys.py and strip trailing slash to ensure correct parsing
botify_project_url = locals().get('botify_project_url', keys.botify_project_url).rstrip('/')
botify_token = keys.botify # Directly use token from keys module
client_domain = keys.client_domain # Explicitly bring client_domain into local scope

# 2. Parse organization and project slugs
try:
    url_parts = botify_project_url.split('/')
    
    # Botify URL structure is .../org_slug/project_slug/
    org = url_parts[-2]      # e.g., 'bare-necessities-org'
    project = url_parts[-1]  # e.g., 'bare-necessities'
    
    print(f"Parsed Org: {org}, Project: {project}")

except Exception as e:
    # SystemExit is appropriate here as API calls with bad slugs will fail
    raise SystemExit(f"Botify URL parsing failed for {botify_project_url}. Check keys.botify_project_url format.")

# 3. Dynamically fetch the latest analysis slug
slugs = fetch_analysis_slugs(org, project, botify_token)

if slugs:
    analysis = slugs[0] # Use the most recent analysis slug
    print(f"✅ Found latest Analysis Slug: {analysis}")
else:
    raise SystemExit("Could not find any Botify analysis slugs for the provided project. Aborting Botify step.")
    
# The working directory for Botify data (consistent location)
csv_dir = Path("data") / f"{job}_botify"
csv_dir.mkdir(parents=True, exist_ok=True)
report_name = csv_dir / "botify_export.csv"

# --- 2. MAIN WORKFLOW LOGIC (API CALLS) ---

# BQLv1 API query: Full payload including GSC data
if not report_name.exists():
    data_payload = {
        "fields": [
            "url", "depth", "gsc_by_url.count_missed_clicks", "gsc_by_url.avg_ctr", 
            "gsc_by_url.avg_position", "inlinks_internal.nb.unique", "internal_page_rank.value", 
            "internal_page_rank.position", "internal_page_rank.raw", "gsc_by_url.count_impressions", 
            "gsc_by_url.count_clicks", "gsc_by_url.count_keywords", 
            "gsc_by_url.count_keywords_on_url_to_achieve_90pc_clicks", 
            "metadata.title.content", "metadata.description.content",
        ],
        "sort": []
    }

    print("Checking if we need to download file from Botify (Full GSC Payload)...")

    # Call the defined export function
    status_code, download_url = export_data(
        version='v1',
        org=org,
        project=project,
        export_payload=data_payload,
        report_path=report_name,
        analysis=analysis # Now correctly populated
    )

    if status_code == 200:
        botify_export_df = pd.read_csv(report_name, skiprows=1)
        print("✅ We have the Botify file (Full Payload).")
    
    elif status_code is not None:
        # --- FALLBACK LOGIC: Try with no GSC Data ---
        print("❌ Full GSC Payload failed. Trying again without GSC data.")
        data_payload_fallback = {
            "fields": [
                "url", "depth", "inlinks_internal.nb.unique", "internal_page_rank.value", 
                "internal_page_rank.position", "internal_page_rank.raw", 
                "metadata.title.content", "metadata.description.content",
            ],
            "sort": []
        }
        status_code, download_url = export_data(
            version='v1',
            org=org,
            project=project,
            export_payload=data_payload_fallback,
            report_path=report_name,
            analysis=analysis
        )
        if status_code == 200:
            botify_export_df = pd.read_csv(report_name, skiprows=1)
            print("✅ We have the Botify file (Fallback Payload).")
        else:
            print("❌ No Botify Data found. No GSC or Internal PageRank columns will appear.")
            botify_export_df = None
    
    else:
        print("❌ Botify export failed critically after both attempts.")
        botify_export_df = None

else:
    # File already exists on disk
    botify_export_df = pd.read_csv(report_name, skiprows=1)
    print("☑️ The Botify export file exists (previously downloaded).\n")

# --- 3. DISPLAY AND PIPELINE UPDATE ---

if botify_export_df is not None:
    print("This will give you an idea of the PageRank opportunity within the site...\n")
    
    # Check for the expected column in the DataFrame
    # Note: Botify API exports often use friendly names, not the API field names.
    if "Internal Pagerank" in botify_export_df.columns:
        display(botify_export_df["Internal Pagerank"].value_counts())
    else:
        print("⚠️ Botify Pagerank column not found in DataFrame for display.")
    
    # Store the result in the pipeline
    pip.set(job, 'botify_export_df_json', botify_export_df.to_json(orient='records'))

    # --- STRIPPED WIDGETS REPLACED WITH TEXT OUTPUT ---
    print(f"\n📁 Botify data saved to: {report_name.resolve()}")
    print(f"📂 Containing folder: {csv_dir.resolve()}")

In [ ]:
import pandas as pd
from pathlib import Path
from IPython.display import display
import numpy as np

# NOTE: This cell assumes 'job', 'arranged_df', 'botify_export_df' (can be None), 
#       and 'unformatted_csv' are available from previous cells.

# --- PATH DEFINITION ---
# The next intermediate file where the user can inspect unformatted data.
unformatted_csv = Path("data") / f"{job}_unformatted.csv"

# --- REQUIRED SUPPORT FUNCTION (Surgically Ported from botifython.py) ---
def insert_columns_after_surgical(df, column_names, after_column):
    """
    Surgical port: Inserts a list of columns immediately after a specified column.
    """
    if after_column not in df.columns:
        # If the reference column is missing, append columns to the end
        new_order = df.columns.tolist() + [col for col in column_names if col not in df.columns]
        return df[new_order]
        
    missing_columns = [col for col in column_names if col not in df.columns]
    
    # Drop columns that are supposed to be inserted but don't exist in the current DataFrame
    # This prevents the ValueError when trying to insert a missing column.
    column_names = [col for col in column_names if col not in missing_columns]

    if not column_names:
        return df # Nothing to insert

    insert_after_index = df.columns.get_loc(after_column)
    
    before = df.columns[:insert_after_index + 1].tolist()
    after = df.columns[insert_after_index + 1:].tolist()
    
    # Ensure columns to be inserted are not duplicated in the 'after' list
    after = [col for col in after if col not in column_names] 
    
    new_order = before + column_names + after
    return df[new_order]


print("Joining Gap Analsys to Extra Botify Columns...")

# The original code implicitly relied on 'arranged_df' having a column named 'URL' 
# and 'botify_export_df' having 'Full URL'.

# 1. Determine if Botify data is present (Handling the optional nature)
if isinstance(botify_export_df, pd.DataFrame) and not botify_export_df.empty:
    has_botify = True
    
    # Perform the merge. The API export uses 'url' but the resulting CSV often labels it 'Full URL'
    # We rename the column in botify_export_df to a standard name for the merge key
    botify_data_for_merge = botify_export_df.rename(columns={"url": "Full URL"}, inplace=False)
    
    # Perform left merge
    final_df = pd.merge(arranged_df, botify_data_for_merge, left_on='Client URL', right_on='Full URL', how='left')

    # Original logic: final_df = bf.insert_columns_after(final_df, list(botify_export_df.columns), "Competition")
    # Transposed logic: Insert Botify columns after the 'Competition' column.
    botify_cols = [col for col in botify_export_df.columns if col not in final_df.columns]
    final_df = insert_columns_after_surgical(final_df, botify_cols, "Competition")
    
    print("Botify Data was found and used.\n")
    
else:
    has_botify = False
    # If no Botify data, the final data frame is just the arranged SEMRush data
    final_df = arranged_df.copy() 
    print("No Botify Data.\n")

# 2. Final Cleanup and Persistence
final_df = final_df.copy()

# Drop columns. Note: 'Client URL' is the join key in the arranged_df, but the 
# logic drops "URL" and "Full URL" and later drops the traffic columns. We respect
# the original's intent to clean up the redundant URL columns used for merging.
if "URL" in final_df.columns:
    final_df.drop(columns=["URL"], inplace=True)
if "Full URL" in final_df.columns:
    final_df.drop(columns=["Full URL"], inplace=True)
    
# Remove columns that often get created as artifacts during processing but are not the final expected columns
if 'url' in final_df.columns:
    final_df.drop(columns=['url'], inplace=True)


# Save unformatted intermediary file (critical side effect)
final_df.to_csv(unformatted_csv, index=False)
df = final_df.copy() # Assign to 'df' as the next working DataFrame

# 3. Display Metrics
rows, cols = final_df.shape
print(f"Rows: {rows:,}")
print(f"Cols: {cols:,}")

# The check for 'Internal Pagerank' presence is only meaningful if Botify data was included
if has_botify and "Internal Pagerank" in final_df.columns:
    display(final_df["Internal Pagerank"].value_counts())
else:
    print("⚠️ Internal Pagerank not available for display.")

# Store the final working DataFrame in the pipeline
pip.set(job, 'final_working_df_json', df.to_json(orient='records'))

In [ ]:
import pandas as pd
from IPython.display import display

# NOTE: This cell assumes 'ROW_LIMIT' (from config) and 'final_df' (from the previous step) are available.

# 1. Define the Constraint
limit_rows = ROW_LIMIT
print(f"✂️ Truncating data to fit under {limit_rows:,} rows for clustering and final deliverable size...")

# 2. Define the Search Volume Cut-off Increments
# These are designed to quickly step toward the target volume constraint.
volume_cutoffs = [49, 99, 199, 299, 499, 999, 1499, 1999, 2499, 2999, 3499, 3999, 5000, 7500, 10000, 20000, 30000]

truncated_df = final_df.copy() # Initialize just in case the loop never runs
try_fit = 0 # Default value if the entire list is under the limit

# 3. Iterate to find the optimal Search Volume floor
for cutoff in volume_cutoffs:
    # Filter: Keep keywords with Search Volume STRICTLY GREATER THAN the cutoff
    df_candidate = final_df[final_df["Search Volume"] > cutoff]
    
    num_rows = df_candidate.shape[0]
    try_fit = cutoff # Update the current cutoff value
    
    print(f"Volume >{cutoff:,} results in {num_rows:,} rows.")
    
    # 4. Check the Breakpoint Condition
    if num_rows <= limit_rows:
        truncated_df = df_candidate # This is the best fit found so far
        break

# If the loop completes without breaking (i.e., the whole initial dataset is under the limit)
# the 'truncated_df' remains the initial copy, and 'try_fit' is the last item.
if truncated_df.shape[0] == 0 and final_df.shape[0] > 0:
    # Handle edge case where first filter cuts everything. Use the last safe cutoff.
    truncated_df = final_df[final_df["Search Volume"] > 0]
    
# 5. Final Output and Persistence
rows, cols = truncated_df.shape
print(f"✅ Final truncation floor: Search Volume >{try_fit:,} resulting in {rows:,} rows.")

# Update the main working DataFrame to the truncated version for all subsequent steps
df = truncated_df.copy()

# Store the truncated DataFrame in the pipeline
pip.set(job, 'truncated_df_for_clustering_json', df.to_json(orient='records'))

display(df.head())

In [ ]:
# --- KEYWORD CLUSTERING SUPPORT FUNCTIONS (REQUIRES: nltk, sklearn, wordninja) ---
from collections import Counter
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
import numpy as np

# --- 1. CORE ML UTILITIES ---

def calculate_silhouette(X, labels):
    """Calculates the Silhouette Coefficient for cluster evaluation."""
    # Handle the edge case where there is only one cluster or too few data points
    if len(np.unique(labels)) <= 1 or len(X) < 2:
        return 0.0 # Return 0 for non-evaluatable cases

    return silhouette_score(X, labels)

def preprocess_keywords(text):
    """Stems, lowercases, tokenizes, and removes stopwords from a keyword string."""
    stemmer = PorterStemmer()
    # Assuming stopwords were downloaded earlier with `nltk.download('stopwords')`
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    # Filter for alphanumeric words and then stem
    return ' '.join([stemmer.stem(word) for word in words if word not in stop_words and word.isalnum()])

def keyword_clustering(df, keyword_column, n_clusters=30, n_components=5, max_features=500):
    """Performs Tfidf Vectorization, Truncated SVD, and MiniBatchKMeans clustering."""

    # 1. Preprocess keywords
    df['Stemmed Keywords'] = df[keyword_column].apply(preprocess_keywords)

    # 2. Text Vectorization
    print(f"Vectorizing... (Max Features: {max_features})")
    vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english')
    X = vectorizer.fit_transform(df['Stemmed Keywords'])

    # 3. Dimension Reduction
    print(f"Reducing Dimensions... (Components: {n_components})")
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    principal_components = svd.fit_transform(X)

    # 4. Clustering
    print(f"Clustering... (K: {n_clusters})")
    # Setting compute_labels=True to ensure compatibility with MiniBatchKMeans
    minibatch_kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=100, random_state=42, n_init='auto') 
    df['Keyword Cluster'] = minibatch_kmeans.fit_predict(principal_components)

    # 5. Calculate silhouette score
    print("Calculating silhouette cluster quality score (takes a bit)...")
    silhouette_avg = calculate_silhouette(principal_components, df['Keyword Cluster'])

    # Return DataFrame, score, and the used parameters
    return df, silhouette_avg, {'n_clusters': n_clusters, 'n_components': n_components, 'max_features': max_features}

def name_keyword_clusters(df, keyword_column, cluster_column):
    """Names each cluster by the most common non-stopword, non-repeating bigram within the cluster."""

    stop_words = set(stopwords.words('english'))
    cluster_names = {}

    for cluster in df[cluster_column].unique():
        cluster_data = df[df[cluster_column] == cluster]
        all_keywords = ' '.join(cluster_data[keyword_column].astype(str)).split()
        filtered_keywords = [word for word in all_keywords if word not in stop_words and word.isalnum()]

        bigram_counts = Counter(bigrams(filtered_keywords))

        most_common_bigram = None
        for bigram, count in bigram_counts.most_common():
            if bigram[0] != bigram[1]:
                most_common_bigram = bigram
                break

        if not most_common_bigram:
            # Fallback to single most common word or a generic name
            unigram_counts = Counter(filtered_keywords)
            most_common_unigram = unigram_counts.most_common(1)
            most_common_words = most_common_unigram[0][0] if most_common_unigram else "Generic Cluster"
        else:
            most_common_words = ' '.join(most_common_bigram)

        cluster_names[cluster] = most_common_words

    df['Keyword Group (Experimental)'] = df[cluster_column].map(cluster_names)

    # Drop Process Columns (as per original logic)
    df.drop(columns=['Stemmed Keywords'], inplace=True)
    df.drop(columns=['Keyword Cluster'], inplace=True)

    return df

# --- REMAINING UTILITIES (as supplied, but not directly called by the main block) ---
# analyze_keyword_frequencies, estimate_clusters_from_frequencies, test_cluster_sizes are not needed in main block

In [ ]:
import itertools
import json
from pathlib import Path
from IPython.display import display
import re # Needed for the reorder function (reorder_columns_surgical)

# NOTE: This cell assumes 'df' (the truncated DataFrame), 'job', and 'has_botify' are defined.

# --- PATH DEFINITIONS ---
# The parameter file is saved alongside other intermediate files in the /data folder
keyword_cluster_params = Path("data") / f"{job}_keyword_cluster_params.json" 
# The final CSV output file path
unformatted_csv = Path("data") / f"{job}_unformatted.csv"

# --- REQUIRED SUPPORT FUNCTION (Re-defined for chaining safety) ---
def reorder_columns_surgical(df, priority_column, after_column):
    """
    Surgical port of bf.reorder_columns: Moves a column immediately after a specified column.
    """
    if priority_column in df.columns and after_column in df.columns:
        columns = df.columns.drop(priority_column).tolist()
        after_column_index = columns.index(after_column)
        columns.insert(after_column_index + 1, priority_column)
        return df[columns]
    elif priority_column not in df.columns:
        print(f"⚠️ Warning: Priority column '{priority_column}' not found for reorder.")
    elif after_column not in df.columns:
        print(f"⚠️ Warning: After column '{after_column}' not found for reorder.")
    return df
    
# --- MAIN CLUSTERING LOGIC ---
print("Grouping Keywords...")

# Download any necessary nltk components (punkt_tab is often missed)
nltk.download('punkt_tab', quiet=True)

# Configuration for iterative testing
target_silhouette_score = 0.6
n_clusters_options = range(15, 26)
n_components_options = [10, 15, 20]
max_features_options = [50, 100, 150]
total_tests = len(list(itertools.product(n_clusters_options, n_components_options, max_features_options)))

best_score = -1.0 # Initialize to a value lower than any possible score
best_params = {}

# 1. Check for Cached Parameters
if keyword_cluster_params.exists():
    try:
        with keyword_cluster_params.open('r') as file:
            best_params = json.load(file)
            print(f"Loaded initial parameters: {best_params}")
            
            # Test with loaded parameters
            df, score, _ = keyword_clustering(df, 'Keyword', **best_params)
            best_score = score
            print(f"Initial test with loaded parameters: Score = {score:.3f}")
    except (json.JSONDecodeError, FileNotFoundError, TypeError, ValueError) as e:
        print(f"⚠️ Failed to load or use cached parameters. Starting full search. Error: {e}")
        best_params = {}


# 2. Iterative Search (if target score is not met)
if best_score < target_silhouette_score:
    print(f"Refining best keyword clustering fit... Total tests: {total_tests}")
    
    # Use a product of options to test every combination
    for n_clusters, n_components, max_features in itertools.product(n_clusters_options, n_components_options, max_features_options):
        
        # Skip if these parameters match the already tested/loaded ones
        if (n_clusters == best_params.get('n_clusters') and
            n_components == best_params.get('n_components') and
            max_features == best_params.get('max_features')):
            continue

        # Run clustering and get new score/params
        df_temp, score, params = keyword_clustering(df.copy(), 'Keyword', n_clusters, n_components, max_features)
        print(f'Testing params: {params}, Score: {score:.3f}')

        if score > best_score:
            best_score = score
            best_params = params
            df = df_temp.copy() # Keep the DataFrame with the better cluster labels

            # Check for early stop condition
            if best_score >= target_silhouette_score:
                print(f'✅ Good enough score found: {best_score:.3f} with params {best_params}')
                with keyword_cluster_params.open('w') as file:
                    json.dump(best_params, file)
                print(f'Saved best parameters: {best_params}')
                break
    
    # If the search finished without hitting the target, save the highest score achieved
    if best_score < target_silhouette_score and best_params:
        print(f'Highest score reached: {best_score:.3f}. Saving best parameters found.')
        with keyword_cluster_params.open('w') as file:
            json.dump(best_params, file)
            
# 3. Finalize Clustering (Ensures the final best DataFrame is used if no search was run)
# This step is crucial if the initial loaded parameters were already above the target, 
# as the 'df' used inside the loop might be a temporary copy. Re-cluster with the best params found.
if 'Keyword Cluster' not in df.columns: # If the original clustering didn't run or was skipped
    df, _, _ = keyword_clustering(df, 'Keyword', **best_params)


# 4. Naming clusters and saving the result
print("\nNaming clusters...")
# Original logic: df = bf.name_keyword_clusters(df, 'Keyword', 'Keyword Cluster')
df = name_keyword_clusters(df, 'Keyword', 'Keyword Cluster')


# --- FINAL REORDERING (Using surgical replacement of bf.reorder_columns) ---
df = reorder_columns_surgical(df, 'CPC', after_column='Keyword Difficulty')
df = reorder_columns_surgical(df, 'Keyword Group (Experimental)', after_column='Number of Words')
df = reorder_columns_surgical(df, 'CPC', after_column='Number of Words') # This reorder is duplicated/redundant with the previous one, but ported verbatim

# Conditional column reordering based on data presence
if has_botify:
    # The diagnostic showed the column is 'Meta Description'. We use this.
    df = reorder_columns_surgical(df, 'Client URL', after_column='Meta Description')
else:
    # Fallback remains unchanged (no Meta Description column to place Client URL after)
    df = reorder_columns_surgical(df, 'Client URL', after_column='Competition')
    
# Original: df = bf.reorder_columns(df, 'Competitor URL', after_column='Client URL')
df = reorder_columns_surgical(df, 'Competitor URL', after_column='Client URL')

# Final file persistence
df.to_csv(unformatted_csv, index=False)


# --- DISPLAY FINAL CLUSTER COUNTS ---
print("\nFinal Keyword Group Counts:")
value_counts = df["Keyword Group (Experimental)"].value_counts()
if not value_counts.empty:
    max_digits = len(str(len(value_counts)))
    max_index_width = max(len(str(index)) for index in value_counts.index)
    max_count_width = max(len(f"{count:,}") for count in value_counts)
    
    for i, (index, count) in enumerate(value_counts.items(), start=1):
        counter_str = str(i).zfill(max_digits)
        count_str = f"{count:,}"
        print(f"{counter_str}: {index:<{max_index_width}} - {count_str:>{max_count_width}}")
else:
    print("❌ No keyword groups were created.")

# Store final DataFrame in the pipeline
pip.set(job, 'final_clustered_df_json', df.to_json(orient='records'))

In [ ]:
# --- CORE SUPPORT FUNCTIONS ---
import pandas as pd
import numpy as np
from pathlib import Path
import os
import platform
import subprocess
import ipywidgets as widgets
from IPython.display import display

# Surgical port of _open_folder from FAQuilizer, necessary for the widget to work
def _open_folder(path_str: str = "."):
    """Opens the specified folder in the system's default file explorer."""
    folder_path = Path(path_str).resolve()
    
    if not folder_path.exists() or not folder_path.is_dir():
        print(f"❌ Error: Path is not a valid directory: {folder_path}")
        return

    system = platform.system()
    try:
        if system == "Windows":
            os.startfile(folder_path)
        elif system == "Darwin":  # macOS
            subprocess.run(["open", folder_path])
        else:  # Linux (xdg-open covers most desktop environments)
            subprocess.run(["xdg-open", folder_path])
    except Exception as e:
        print(f"❌ Failed to open folder. Error: {e}")

# This utility must be defined for normalize_and_score to work
def safe_normalize(series):
    """ Normalize the series safely to avoid divide by zero and handle NaN values. """
    min_val = series.min()
    max_val = series.max()
    range_val = max_val - min_val
    if range_val == 0:
        # Avoid division by zero by returning zero array if no range
        return np.zeros_like(series)
    else:
        # Normalize and fill NaN values that might result from empty/NaN series
        return (series - min_val).div(range_val).fillna(0)

# Surgical port of bf.reorder_columns
def reorder_columns_surgical(df, priority_column, after_column):
    if priority_column in df.columns:
        columns = list(df.columns.drop(priority_column))
        # Handle cases where the after_column may have been dropped earlier
        if after_column not in columns:
            print(f"⚠️ Reorder Error: Target column '{after_column}' not found. Skipping reorder of '{priority_column}'.")
            return df
            
        after_column_index = columns.index(after_column)
        columns.insert(after_column_index + 1, priority_column)
        df = df[columns]
    else:
        print(f"⚠️ Reorder Error: Column '{priority_column}' not found in DataFrame.")
    return df

# Surgical port of bf.normalize_and_score - WITH CRITICAL FIX
def normalize_and_score_surgical(df, registered_domain, has_botify_data, after_col, reorder):
    
    # Rename original column fields to match expected names in the dataframe
    if 'internal_page_rank.raw' in df.columns:
        df = df.rename(columns={'internal_page_rank.raw': 'Raw Internal Pagerank'}, inplace=False)
    
    # --- CRITICAL FIX FOR KEYERROR / TRAILING SLASHES ---
    # The lookup key (e.g., 'nixos.org') must be matched against the DataFrame column (e.g., 'nixos.org/').
    # We clean both for comparison to find the unique canonical key, but use the original column name.
    
    # Clean the lookup domain (assuming the input `registered_domain` might be missing the slash)
    clean_lookup_key = registered_domain.rstrip('/')
    target_col = None
    
    for col in df.columns:
        # Find the column whose stripped name matches the stripped lookup key.
        if col.rstrip('/') == clean_lookup_key:
            target_col = col
            break
            
    if target_col is None:
        raise KeyError(f"Could not find client domain column for '{registered_domain}' in DataFrame. Available columns: {df.columns.tolist()}")
    # --- END CRITICAL FIX ---
    
    # Normalize metrics that are always included
    df['Normalized Search Volume'] = safe_normalize(df['Search Volume'])
    df['Normalized Search Position'] = safe_normalize(df[target_col]) # <-- USES THE FOUND, CANONICAL COLUMN NAME
    df['Normalized Keyword Difficulty'] = safe_normalize(df['Keyword Difficulty'])
    df['Normalized CPC'] = safe_normalize(df['CPC'])

    # Always include CPC and Keyword Difficulty in the combined score
    combined_score = df['Normalized CPC'] - df['Normalized Keyword Difficulty']

    if has_botify_data:
        # Normalize additional Botify metrics if available
        if 'Raw Internal Pagerank' in df.columns:
            df['Normalized Raw Internal Pagerank'] = safe_normalize(df['Raw Internal Pagerank'])
        else:
            df['Normalized Raw Internal Pagerank'] = 0

        if "No. of Missed Clicks excluding anonymized queries" in df.columns:
            df['Normalized Missed Clicks'] = safe_normalize(df["No. of Missed Clicks excluding anonymized queries"])
            combined_score += df['Normalized Missed Clicks']
        else:
            df['Normalized Missed Clicks'] = 0

        # Add Botify metrics to the combined score
        combined_score += (-1 * df['Normalized Raw Internal Pagerank'] +
                           df['Normalized Search Volume'] +
                           df['Normalized Search Position'])

    # Apply the combined score to the DataFrame
    df['Combined Score'] = combined_score

    if reorder:
        # Reorder columns if required (using the surgically ported reorder function)
        df = reorder_columns_surgical(df, "CPC", after_col)
        df = reorder_columns_surgical(df, "Keyword Difficulty", "CPC")
        if has_botify_data:
            df = reorder_columns_surgical(df, "Internal Pagerank", "Keyword Difficulty")
            df = reorder_columns_surgical(df, "No. of Unique Inlinks", "Internal Pagerank")
            if "No. of Missed Clicks excluding anonymized queries" in df.columns:
                df = reorder_columns_surgical(df, "No. of Missed Clicks excluding anonymized queries", "No. of Unique Inlinks")

    return df

In [ ]:
import pandas as pd
import xlsxwriter
import itertools
from pathlib import Path
from IPython.display import display
import os
import platform
import subprocess
import ipywidgets as widgets
# NOTE: This cell assumes 'job', 'df', 'competitors', 'semrush_lookup', 'has_botify' are defined.

# --- 1. DEFINE SECURE OUTPUT PATHS ---
# Create the secure, client-specific deliverables folder: Notebooks/deliverables/{job}/
deliverables_dir = Path("deliverables") / job
deliverables_dir.mkdir(parents=True, exist_ok=True)

# Define the final Excel file path (using a standardized, clean name for the file)
# We ensure the filename is clean by stripping the trailing slash/underscore that might be left by SEMRush input.
xl_filename = f"{semrush_lookup.replace('.', '_').rstrip('_')}_GAPalyzer_{job}_V1.xlsx"
xl_file = deliverables_dir / xl_filename

# --- 2. EXECUTE CORE LOGIC ---
print(f"- Writing Gap Analysis tab to {xl_file.name} (first pass)...")

# Initialize Loop List (verbatim)
loop_list = ["Gap Analysis"]
last_competitor = competitors[-1]

# Apply the normalization and scoring logic
# The target_col lookup inside this function will now succeed.
df_tab = normalize_and_score_surgical(df.copy(), semrush_lookup, has_botify, last_competitor, False)

# Save Initial Excel Sheet with xlsxwriter (Engine Kwargs included verbatim)
arg_dict = {'options': {'strings_to_urls': False}}
try:
    with pd.ExcelWriter(xl_file, engine="xlsxwriter", engine_kwargs=arg_dict, mode='w') as writer:
        # NOTE: Using df_tab here, which has the Normalized columns
        df_tab.to_excel(writer, sheet_name='Gap Analysis', index=False)
    
    print("✅ Gap Analysis tab written (Unformatted Pass 1).")
    
except Exception as e:
    print(f"❌ Error writing Excel file: {e}")
    
# --- 3. DISPLAY SECURE EGRESS BUTTON ---

# The button description clearly indicates the file and the security of the folder.
button = widgets.Button(
    description=f"📂 Open Deliverables Folder ({job})",
    tooltip=f"Open {deliverables_dir.resolve()}",
    button_style='success'
)
# Use a lambda function to call the portable _open_folder function on click
button.on_click(lambda b: _open_folder(str(deliverables_dir)))
display(button)

# Store final file path in pipeline
pip.set(job, 'final_xl_file', str(xl_file))
pip.set(job, 'deliverables_folder', str(deliverables_dir))

In [ ]:
print(f"- Adding filter tabs to {xl_file.name} (second pass)...")

import pandas as pd
import xlsxwriter
import openpyxl
import re
from pathlib import Path
from collections import Counter
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np

# NOTE: This cell assumes 'job', 'df', 'semrush_lookup', 'has_botify', 
#       'xl_file', 'competitors', and helper functions are defined.

# --- PATH DEFINITION ---
important_keywords_file = Path("data") / f"{job}_important_keywords.txt" 

# --- CRITICAL FIX START: ENFORCE CANONICAL KEY ---
# The lookup key (semrush_lookup) might be clean ('nixos.org'), while the column name is dirty ('nixos.org/').
# We find the actual dirty column name once, and use that variable (TARGET_COMPETITOR_COL) everywhere.
# NOTE: This logic must match the internal search in normalize_and_score_surgical.

clean_lookup_key = semrush_lookup.rstrip('/')
TARGET_COMPETITOR_COL = None

for col in df.columns:
    if col.rstrip('/') == clean_lookup_key:
        TARGET_COMPETITOR_COL = col
        break

if TARGET_COMPETITOR_COL is None:
    # If the lookup fails here, the process MUST fail gracefully to prevent subsequent runtime errors.
    raise KeyError(f"CRITICAL ERROR: Could not find canonical competitor column for '{semrush_lookup}' in DataFrame. Available columns: {df.columns.tolist()}")

print(f"✅ Canonical Competitor Column Identified: '{TARGET_COMPETITOR_COL}'")
# --- CRITICAL FIX END ---


# --- REQUIRED SUPPORT FUNCTIONS (No changes needed, but must be present) ---
# ... (All helper functions: reorder_columns_surgical, safe_normalize, normalize_and_score_surgical) ...
# Assuming the necessary helper functions are either imported or defined above this block.
# We trust that the version of normalize_and_score_surgical already has the internal fix.


def read_keywords(file_path):
    """Function to read keywords from a file."""
    if not file_path.exists():
        print(f"⚠️ Warning: Keywords file not found at {file_path}. Skipping file-based filter.")
        return []
    with open(file_path, 'r') as file:
        important_keywords_list = [line.strip() for line in file.readlines()]
    return important_keywords_list
    
def filter_df_by_keywords(df, keywords):
    """Function to filter dataframe based on an exact list of keywords."""
    return df[df["Keyword"].isin(keywords)]


# --- MAIN TAB GENERATION LOGIC ---

print("Starting subsequent Excel tab generation (Appending via openpyxl)...")

# --- 1. Filter: Important Keywords (File-Based Filter) ---
filter_name = "Important Keywords"

if important_keywords_file.exists():
    important_keywords_list = read_keywords(important_keywords_file)
    if important_keywords_list:
        print(f"- Writing {filter_name} tab (via file list)...")
        
        df_filtered = filter_df_by_keywords(df.copy(), important_keywords_list)
        
        df_filtered = normalize_and_score_surgical(df_filtered, semrush_lookup, has_botify, competitors[-1], True)
        df_filtered.sort_values(by='Combined Score', ascending=False, inplace=True)
        
        with pd.ExcelWriter(xl_file, engine="openpyxl", mode='a', if_sheet_exists='replace') as writer:
            df_filtered.to_excel(writer, sheet_name=filter_name, index=False)
    else:
        print(f"☑️ Skipping '{filter_name}': Keyword file is empty.")
else:
    print(f"☑️ Skipping '{filter_name}': Keywords file does not exist.")


# --- 2. Filter: Best Opportunities / Striking Distance ---
filter_name = "Best Opportunities"
striking_lower = 100

df_tab = df.copy()

if has_botify:
    try:
        # FIXED LINE: Uses TARGET_COMPETITOR_COL for direct DataFrame indexing
        df_tab = df_tab[(df_tab["No. of Impressions excluding anonymized queries"] > 0) & (df_tab[TARGET_COMPETITOR_COL] > 3)].copy()
        print(f"- Writing {filter_name} tab (Botify/GSC Striking Distance)...")
    except KeyError:
        # Fallback uses TARGET_COMPETITOR_COL
        # FIXED LINE: Uses TARGET_COMPETITOR_COL for fallback indexing
        df_tab = df[(df[TARGET_COMPETITOR_COL] >= 4) & (df[TARGET_COMPETITOR_COL] <= striking_lower)].copy()
        print(f"- Writing {filter_name} tab (SEMRush Striking Distance fallback)...")
else:
    # SEMRush-only logic uses TARGET_COMPETITOR_COL
    # FIXED LINE: Uses TARGET_COMPETITOR_COL
    df_tab = df[(df[TARGET_COMPETITOR_COL] >= 4) & (df[TARGET_COMPETITOR_COL] <= striking_lower)].copy()
    print(f"- Writing {filter_name} tab (SEMRush Striking Distance)...")
    
# Apply scoring, reorder, and sort
df_tab = normalize_and_score_surgical(df_tab.copy(), semrush_lookup, has_botify, competitors[-1], True)
df_tab.sort_values(by='Combined Score', ascending=False, inplace=True)

# Write out the tab
with pd.ExcelWriter(xl_file, engine="openpyxl", mode='a', if_sheet_exists='replace') as writer:
    df_tab.to_excel(writer, sheet_name=filter_name, index=False)


# --- 3. Filter: Important Keywords disable (Client Ranking Sort) ---
filter_name = "Important Keywords disable"

# Filter: Keywords where the client is ranking (position is not null)
# FIXED LINE: Uses TARGET_COMPETITOR_COL
df_tab = df[df[TARGET_COMPETITOR_COL].notnull()].sort_values(by=[TARGET_COMPETITOR_COL, 'Search Volume'], ascending=[True, False]).copy()
    
print(f"- Writing {filter_name} tab (Client Rank Sort)...")
# Write out the tab
with pd.ExcelWriter(xl_file, engine="openpyxl", mode='a', if_sheet_exists='replace') as writer:
    df_tab.to_excel(writer, sheet_name=filter_name, index=False)


# --- 4. Loop: Targeted Keyword Filters (Gifts, Questions, Near Me) ---

targeted_filters = [
    ("Gifts", [
        'gift', 'gifts', 'idea', 'ideas', 'present', 'presents', 'give', 'giving', 
        'black friday', 'cyber monday', 'cyber week', 'bfcm', 'bf', 'cm', 
        'holiday', 'deals', 'sales', 'offer', 'discount', 'shopping'
    ]),
    ("Broad Questions", '''
        am are can could did do does for from had has have how i is may might must shall should was were what when where which who whom whose why will with would
        '''.split()),
    ("Narrow Questions", '''
        who whom whose what which where when why how
        '''.split()),
    ("Popular Modifiers", [
        'how to', 'best', 'review', 'reviews'
    ]),
    ("Near Me", ['near me', 'for sale', 'nearby', 'closest', 'near you', 'local'])
]

for filter_name, keywords in targeted_filters:
    print(f"- Writing {filter_name} tab...")
    
    # Construct the RegEx pattern using word boundaries for precise matching
    pattern = r'\b(?:' + '|'.join([re.escape(k) for k in keywords]) + r')\b'
    
    # Filter 1: Find keywords matching the pattern (e.g., 'gift', 'how', 'near me')
    df_tab = df[df["Keyword"].str.contains(pattern, case=False, na=False)].copy()
    
    # Filter 2: Apply Striking Distance Logic 
    # Keep keywords where client is ranking (pos >= 4) OR client is not ranking at all (NaN)
    # FIXED LINE: Uses TARGET_COMPETITOR_COL
    # df_tab = df_tab[(df_tab[TARGET_COMPETITOR_COL] >= 4) | (df_tab[TARGET_COMPETITOR_COL].isna())]
    
    # Apply scoring, reorder, and sort
    df_tab = normalize_and_score_surgical(df_tab.copy(), semrush_lookup, has_botify, competitors[-1], True)
    df_tab.sort_values(by='Combined Score', ascending=False, inplace=True)
    
    # Write out the tab
    with pd.ExcelWriter(xl_file, engine="openpyxl", mode='a', if_sheet_exists='replace') as writer:
        df_tab.to_excel(writer, sheet_name=filter_name, index=False)


print("✅ Done writing all supplementary Excel tabs.")
# Use a lambda function to call the portable _open_folder function on click
button.on_click(lambda b: _open_folder(str(deliverables_dir)))
display(button)

In [ ]:
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils import get_column_letter
import re # Needed for is_safe_url
import validators # Need to import validators for URL check

print(f"🎨 Applying Excel Formatting to all data tabs in {xl_file.name} (third pass)...")

# NOTE: This cell assumes 'xl_file', 'competitors', 'semrush_lookup', 'has_botify'
#       'TARGET_COMPETITOR_COL' (the verified column name) are defined in previous cells.

# --- REQUIRED SUPPORT FUNCTIONS (Surgically Ported/Defined) ---

def create_column_mapping(sheet):
    """Creates a dictionary mapping header names to column letters."""
    mapping = {}
    for col_idx, column_cell in enumerate(sheet[1], 1): # Assumes headers are in row 1
        column_letter = get_column_letter(col_idx)
        mapping[str(column_cell.value)] = column_letter
    return mapping

def apply_fill_to_column_labels(sheet, column_mapping, columns_list, fill):
    """Applies a fill color to the header cells of specified columns."""
    for column_name in columns_list:
        column_letter = column_mapping.get(column_name)
        if column_letter:
            cell = sheet[f"{column_letter}1"]
            cell.fill = fill

def find_last_data_row(sheet, keyword_column_letter):
    """Finds the last row containing data in a specific column (e.g., 'Keyword')."""
    if not keyword_column_letter: # Handle case where keyword column might be missing
        return sheet.max_row

    last_row = sheet.max_row
    # Iterate backwards from the max row
    while last_row > 1 and sheet[f"{keyword_column_letter}{last_row}"].value in [None, "", " "]:
        last_row -= 1
    return last_row

def apply_conditional_formatting(sheet, column_mapping, last_row, conditionals_descending, conditionals_ascending, rule_desc, rule_asc):
    """Applies color scale conditional formatting to specified columns."""
    for label in conditionals_descending + conditionals_ascending:
        column_letter = column_mapping.get(label)
        if column_letter and last_row > 1: # Ensure there is data to format
            range_string = f'{column_letter}2:{column_letter}{last_row}'
            rule = rule_desc if label in conditionals_descending else rule_asc
            try:
                sheet.conditional_formatting.add(range_string, rule)
            except Exception as e:
                print(f"⚠️ Failed to apply conditional formatting for {label}: {e}")

def is_safe_url(url):
    """ Check if the given string is a valid URL using the validators library. """
    if not isinstance(url, str):
        return False
    # Use validators library for robust URL check
    return validators.url(url)

# Color schemes and patterns
green = '33FF33'
client_color = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid') # Yellow
competitor_color = PatternFill(start_color='EEECE2', end_color='EEECE2', fill_type='solid') # Light Gray
semrush_color = PatternFill(start_color='FAEADB', end_color='FAEADB', fill_type='solid') # Light Orange
semrush_opportunity_color = PatternFill(start_color='F1C196', end_color='F1C196', fill_type='solid') # Darker Orange
botify_color = PatternFill(start_color='EADFF2', end_color='EADFF2', fill_type='solid') # Light Purple
botify_opportunity_color = PatternFill(start_color='AEA1C4', end_color='AEA1C4', fill_type='solid') # Darker Purple
color_scale_rule_desc = ColorScaleRule(start_type='min', start_color='FFFFFF', end_type='max', end_color=green) # White to Green (Higher is Better)
color_scale_rule_asc = ColorScaleRule(start_type='min', start_color=green, end_type='max', end_color='FFFFFF') # Green to White (Lower is Better)

# Create a border style (Subtle hair lines, thin bottom for headers)
thin_border = Border(left=Side(style='hair'), right=Side(style='hair'), top=Side(style='hair'), bottom=Side(style='thin'))

# Commonly reused column widths
tiny_width = 11
small_width = 15
medium_width = 20
description_width = 50
url_width = 70 # Adjusted slightly down from 100 for better viewability

# Define column widths (Verbatim)
column_widths = {
    'Keyword': 40, 'Search Volume': small_width, 'Number of Words': tiny_width,
    'Keyword Group (Experimental)': small_width, 'Competitors Positioning': tiny_width,
    'CPC': tiny_width, 'Keyword Difficulty': tiny_width, 'Competition': tiny_width,
    'Depth': tiny_width, 'No. of Keywords': tiny_width,
    'No. of Impressions excluding anonymized queries': small_width,
    'No. of Clicks excluding anonymized queries': small_width,
    'No. of Missed Clicks excluding anonymized queries': small_width,
    'Avg. URL CTR excluding anonymized queries': tiny_width,
    'Avg. URL Position excluding anonymized queries': tiny_width,
    'No. of Keywords for the URL To Achieve 90% Audience': tiny_width,
    'Raw Internal Pagerank': small_width, 'Internal Pagerank': tiny_width,
    'Internal Pagerank Position': tiny_width, 'No. of Unique Inlinks': tiny_width,
    'Title': description_width, 'Meta Description': description_width,
    'Timestamp': 12, 'SERP Features by Keyword': description_width,
    'Keyword Intents': medium_width, 'Position Type': small_width,
    'Number of Results': medium_width, 'Competitor URL': url_width,
    'Client URL': url_width, # This gets renamed later
    # Normalized/Score columns
    'Normalized CPC': tiny_width, 'Normalized Keyword Difficulty': tiny_width,
    'Normalized Raw Internal Pagerank': tiny_width, 'Normalized Search Volume': tiny_width,
    'Normalized Search Position': tiny_width, 'Normalized Missed Clicks': tiny_width,
    'Combined Score': tiny_width
}

# Commonly used number formats (Verbatim)
int_fmt = '0'
comma_fmt = '#,##0'
pct_fmt = '0.00'
date_fmt = 'yyyy-mm-dd' # Added for Timestamp clarity

# Define number formats (Added Timestamp)
number_formats = {
    'Search Volume': comma_fmt, 'Number of Words': int_fmt, 'CPC': pct_fmt,
    'Keyword Difficulty': int_fmt, 'Competition': pct_fmt, 'Depth': int_fmt,
    'No. of Keywords': comma_fmt, 'No. of Impressions excluding anonymized queries': comma_fmt,
    'No. of Clicks excluding anonymized queries': comma_fmt,
    'No. of Missed Clicks excluding anonymized queries': comma_fmt,
    'Avg. URL CTR excluding anonymized queries': pct_fmt,
    'Avg. URL Position excluding anonymized queries': '0.0',
    'No. of Keywords for the URL To Achieve 90% Audience': comma_fmt,
    'Raw Internal Pagerank': '0.0000000', 'Internal Pagerank': pct_fmt,
    'Internal Pagerank Position': int_fmt, 'No. of Unique Inlinks': comma_fmt,
    'Number of Results': comma_fmt, 'Timestamp': date_fmt,
    # Apply comma format to positioning and scores for consistency
    'Competitors Positioning': int_fmt, 'Normalized CPC': pct_fmt,
    'Normalized Keyword Difficulty': pct_fmt, 'Normalized Raw Internal Pagerank': pct_fmt,
    'Normalized Search Volume': pct_fmt, 'Normalized Search Position': pct_fmt,
    'Normalized Missed Clicks': pct_fmt, 'Combined Score': '0.00'
}

# --- DEFINE COLUMN GROUPS FOR COLORING (Verbatim, adapted for known columns) ---
# Higher Numbers More Green (Descending is better)
conditionals_descending = [
    'Search Volume', 'CPC', 'Competition', # Removed Traffic metrics as they were dropped
    'Avg. URL CTR excluding anonymized queries',
    'No. of Missed Clicks excluding anonymized queries', 'Combined Score',
    'No. of Unique Inlinks' # Added Inlinks (usually higher is better contextually)
]
# Lower Numbers More Green (Ascending is better)
conditionals_ascending = [
    'Keyword Difficulty', 'Raw Internal Pagerank', 'Internal Pagerank',
    'Internal Pagerank Position', 'Avg. URL Position excluding anonymized queries', 'Depth',
    TARGET_COMPETITOR_COL # Add the client's position column dynamically
] + [col for col in competitors if col != TARGET_COMPETITOR_COL] # Add other competitor position columns

# SEMRush Data Columns
semrush_columns = [
    'Keyword', 'Search Volume', 'CPC', 'Keyword Difficulty', 'Competition',
    'SERP Features by Keyword', 'Keyword Intents', 'Position Type',
    'Number of Results', 'Timestamp', 'Competitor URL', 'Client URL' # Includes Client URL before rename
]
# Botify Data Columns (Ensure these match final DataFrame after merge)
botify_columns = [
    'Depth', 'No. of Keywords', 'No. of Impressions excluding anonymized queries',
    'No. of Clicks excluding anonymized queries', 'No. of Missed Clicks excluding anonymized queries',
    'Avg. URL CTR excluding anonymized queries', 'Avg. URL Position excluding anonymized queries',
    'No. of Keywords for the URL To Achieve 90% Audience', 'Raw Internal Pagerank',
    'Internal Pagerank', 'Internal Pagerank Position', 'No. of Unique Inlinks',
    'Title', 'Meta Description' # Changed from API name
]
# Columns which get bigger header fonts
bigger_font_headers = [
    "Keyword", "Search Volume", "Title", "Meta Description",
    "Competitor URL", "Client URL", "SERP Features by Keyword"
]
# Columns which get darker Botify color
botify_opportunity_columns = [
    'Internal Pagerank', 'No. of Unique Inlinks',
    'No. of Missed Clicks excluding anonymized queries',
    'Normalized Raw Internal Pagerank', 'Normalized Missed Clicks'
]
# Columns which get darker SEMRush color
semrush_opportunity_columns = [
    'CPC', 'Keyword Difficulty', 'Normalized CPC', 'Normalized Keyword Difficulty',
    'Normalized Search Volume', 'Normalized Search Position', 'Combined Score' # Added Combined Score here
]


# --- APPLY FORMATTING TO EXCEL FILE ---
try:
    wb = load_workbook(xl_file)

    # --- UPDATED: Get all sheet names EXCEPT the diagnostics sheet ---
    sheets_to_format = [name for name in wb.sheetnames if name != "Filter Diagnostics"]
    # -----------------------------------------------------------------

    if not sheets_to_format:
         print("⚠️ No data sheets found in the Excel file to format. Skipping formatting.")

    for sheet_name in sheets_to_format:
        print(f"- Formatting '{sheet_name}' tab...")
        sheet = wb[sheet_name]
        column_mapping = create_column_mapping(sheet)

        # Determine the last row with data based on the 'Keyword' column
        keyword_col_letter = column_mapping.get("Keyword")
        # Add a check in case a sheet somehow doesn't have a Keyword column
        if not keyword_col_letter:
             print(f"  Skipping sheet '{sheet_name}': Cannot find 'Keyword' column for formatting reference.")
             continue
             
        last_row = find_last_data_row(sheet, keyword_col_letter)
        
        # --- Apply Formatting ---

        # 1. Fill client column (using TARGET_COMPETITOR_COL identified earlier)
        client_column_letter = column_mapping.get(TARGET_COMPETITOR_COL)
        if client_column_letter:
            for row in range(1, last_row + 1):
                cell = sheet[f"{client_column_letter}{row}"]
                cell.fill = client_color
                if row == 1: cell.font = Font(bold=True) # Bold header

        # 2. Fill Header Backgrounds
        apply_fill_to_column_labels(sheet, column_mapping, semrush_columns, semrush_color)
        apply_fill_to_column_labels(sheet, column_mapping, botify_columns, botify_color)
        # Apply competitor color only to competitor columns *present* in this sheet
        present_competitors = [c for c in competitors if c in column_mapping and c != TARGET_COMPETITOR_COL]
        apply_fill_to_column_labels(sheet, column_mapping, present_competitors, competitor_color)
        apply_fill_to_column_labels(sheet, column_mapping, botify_opportunity_columns, botify_opportunity_color)
        apply_fill_to_column_labels(sheet, column_mapping, semrush_opportunity_columns, semrush_opportunity_color)

        # 3. Header Styling (Alignment, Font, Border)
        header_font = Font(bold=True)
        header_align = Alignment(horizontal='center', vertical='center', wrap_text=True)
        for header, col_letter in column_mapping.items():
            cell = sheet[f"{col_letter}1"]
            cell.alignment = header_align
            cell.font = header_font
            cell.border = thin_border # Apply border to header
            if header in bigger_font_headers:
                 cell.font = Font(size=14, bold=True) # Slightly smaller than original for balance

        # 4. Hyperlinks (Competitor URL, Client URL)
        for col_label in ["Competitor URL", "Client URL"]:
            col_letter = column_mapping.get(col_label)
            if col_letter:
                for row in range(2, last_row + 1):
                    cell = sheet[f"{col_letter}{row}"]
                    url = cell.value
                    if url and is_safe_url(url) and not str(url).startswith('=HYPERLINK'):
                        # Truncate displayed URL if very long, keep full URL in link
                        display_text = url if len(url) <= 80 else url[:77] + "..."
                        cell.value = f'=HYPERLINK("{url}", "{display_text}")'
                        cell.font = Font(color="0000FF", underline="single")
                        cell.alignment = Alignment(vertical='top', wrap_text=False) # Prevent wrap for URLs


        # 5. Rotate Competitor Headers & Set Width
        competitor_header_align = Alignment(vertical='bottom', textRotation=90, horizontal='center')
        for competitor_col_name in competitors:
            col_letter = column_mapping.get(competitor_col_name)
            if col_letter:
                cell = sheet[f"{col_letter}1"]
                cell.alignment = competitor_header_align
                sheet.column_dimensions[col_letter].width = 4

        # 6. Apply Column Widths (with Global Adjustment)
        for label, width in column_widths.items():
            column_letter = column_mapping.get(label)
            if column_letter:
                # Apply the global width adjustment multiplier
                sheet.column_dimensions[column_letter].width = width * GLOBAL_WIDTH_ADJUSTMENT

        # 7. Apply Number Formats
        for label, format_code in number_formats.items():
            column_letter = column_mapping.get(label)
            if column_letter:
                for row in range(2, last_row + 1):
                    cell = sheet[f"{column_letter}{row}"]
                    # Apply only if cell is not empty, prevents formatting issues
                    if cell.value is not None:
                        cell.number_format = format_code

        # 8. Apply Conditional Formatting (Using the combined rules)
        apply_conditional_formatting(sheet, column_mapping, last_row, conditionals_descending, conditionals_ascending, color_scale_rule_desc, color_scale_rule_asc)

        # 9. Rename 'Client URL' Header Dynamically
        client_url_column_letter = column_mapping.get("Client URL")
        if client_url_column_letter:
            header_cell = sheet[f"{client_url_column_letter}1"]
            header_cell.value = f"{TARGET_COMPETITOR_COL} URL" # Use the canonical name

        # 10. Data Cell Alignment (Wrap text, top align)
        data_align = Alignment(wrap_text=False, vertical='top')
        url_columns = [column_mapping.get("Competitor URL"), column_mapping.get("Client URL")] # Get letters before loop
        for row_idx in range(2, last_row + 1):
            for col_idx in range(1, sheet.max_column + 1):
                cell = sheet.cell(row=row_idx, column=col_idx)
                col_letter = get_column_letter(col_idx)
                # Apply default alignment, skip URL columns handled earlier
                if col_letter not in url_columns:
                    cell.alignment = data_align


        # 11. Header Row Height & Freeze Panes
        # Use the explicit configuration variable for header height
        sheet.row_dimensions[1].height = locals().get('max_length', 15) * 9 if 'max_length' in locals() else 100
        sheet.freeze_panes = 'C2' # Freeze panes more appropriately after Keyword/Volume

        # 12. Apply AutoFilter
        max_col_letter = get_column_letter(sheet.max_column)
        if last_row > 0: # Ensure there are rows to filter
             sheet.auto_filter.ref = f"A1:{max_col_letter}{last_row}"

        # 13. (Optional but recommended) Add Table for banded rows (replaces manual banding)
        if last_row > 0: # Ensure there is data for the table
            table_range = f"A1:{max_col_letter}{last_row}"
            table_name = f"DataTable_{re.sub(r'[^A-Za-z0-9_]', '', sheet_name)}" # Sanitize sheet name for table name

            # --- CORRECTED TABLE CHECK ---
            # Defensively check if items in sheet._tables have a .name attribute
            existing_table_names = [t.name for t in sheet._tables if hasattr(t, 'name')]
            if table_name not in existing_table_names:
            # --- END CORRECTION ---
                 tab = Table(displayName=table_name, ref=table_range)
                 style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                                       showLastColumn=False, showRowStripes=True, showColumnStripes=False)
                 tab.tableStyleInfo = style
                 try:
                      sheet.add_table(tab)
                 except ValueError as ve:
                      print(f"  Note: Could not add Excel Table '{table_name}' to sheet '{sheet_name}'. Maybe overlaps existing table? Error: {ve}")
            # Optional: Add an else here if you want to log that the table already exists
            # else:
            #    print(f"  Skipping table creation: Table '{table_name}' already exists in sheet '{sheet_name}'.")

    # Save the workbook with all formatting applied
    wb.save(xl_file)
    print(f"✅ Formatting applied to all data tabs and saved to {xl_file.name}")

except FileNotFoundError:
    print(f"❌ Error: Excel file not found at {xl_file}. Cannot apply formatting.")
except KeyError as e:
     print(f"❌ Error during formatting: A required column key was not found: {e}. Check DataFrame structure.")
     # Safely attempt to print mapping if it exists
     if 'column_mapping' in locals(): print(f"   Column Mapping: {column_mapping}")
except Exception as e:
    print(f"❌ An unexpected error occurred during Excel formatting: {e}")
# Use a lambda function to call the portable _open_folder function on click
button.on_click(lambda b: _open_folder(str(deliverables_dir)))
display(button)